### 2.4 PostgreSQL -  Données composites

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

<div style="font-size:120%" id="create_type">
Création de types composites :
</div>

In [2]:
c.execute('DROP TYPE IF EXISTS person_type CASCADE')
c.execute('DROP TYPE IF EXISTS address_type CASCADE')

sql = '''CREATE TYPE person_type AS (
    first_name  VARCHAR(128),
    last_name   VARCHAR(128)
);'''
c.execute(sql)

sql = '''CREATE TYPE address_type AS (
    number  INTEGER,
    street  VARCHAR(128),
    zipcode INTEGER,
    city    VARCHAR(128)
);'''
c.execute(sql)

conn.commit()

<div style="font-size:120%" id="create_table">
Création d'une table comportant des types composites:
</div>

In [5]:
c.execute('DROP TABLE IF EXISTS students CASCADE')

sql = '''CREATE TABLE students (
    id      SERIAL PRIMARY KEY,
    person  person_type,
    address address_type
);'''
c.execute(sql)

conn.commit()

<div style="font-size:120%" id="insert">
Insertion dans une table comportant des types composites
</div>

In [6]:
c.execute('DELETE FROM students;')

# SQL natif
sql = '''INSERT INTO students (person,address) VALUES (
    ('Raymond','Deubaze'),
    (31,'avenue Guy de Collongue',69130,'Ecully')
);'''
c.execute(sql)
conn.commit()

<div id="insert_dbapi"/>

In [7]:
# avec arguments
sql = 'INSERT INTO students (person,address) VALUES ((%s,%s),(%s,%s,%s,%s))'
args = ('Elsa','Plique',51,'chemin des Mouilles',69130,'Ecully')
c.execute(sql,args)
conn.commit()

In [8]:
# variante
sql = 'INSERT INTO students (person,address) VALUES (%s,%s)'
args = (('Jean','Bombeur'),(51,'chemin des Mouilles',69130,'Ecully'))
c.execute(sql,args)
conn.commit()

<div style="font-size:120%" id="select">
Requêtage dans une table comportant des types composites
</div>

In [9]:
sql = "SELECT * FROM students WHERE (person).first_name = 'Raymond'"
c.execute(sql)
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

1, (Raymond,Deubaze), (31,"avenue Guy de Collongue",69130,Ecully)


In [10]:
sql = "SELECT DISTINCT (address).street FROM students WHERE (address).city = 'Ecully'"
c.execute(sql)
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

avenue Guy de Collongue
chemin des Mouilles


In [11]:
sql = "SELECT * FROM students ORDER BY((person).last_name,(person).first_name)"
c.execute(sql)
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

3, (Jean,Bombeur), (51,"chemin des Mouilles",69130,Ecully)
1, (Raymond,Deubaze), (31,"avenue Guy de Collongue",69130,Ecully)
2, (Elsa,Plique), (51,"chemin des Mouilles",69130,Ecully)


In [12]:
sql = "SELECT (person).first_name, (address).city FROM students"
c.execute(sql)
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

Raymond, Ecully
Elsa, Ecully
Jean, Ecully


In [13]:
sql = "SELECT (person).* FROM students WHERE (address).city = 'Ecully'"
c.execute(sql)
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

Raymond, Deubaze
Elsa, Plique
Jean, Bombeur


In [14]:
sql = "SELECT (person) FROM students WHERE (address).city = 'Ecully'"
c.execute(sql)
for d in c.fetchall(): print(dict(d))

{'person': '(Raymond,Deubaze)'}
{'person': '(Elsa,Plique)'}
{'person': '(Jean,Bombeur)'}


<div style="font-size:120%" id="update">
Modification de la valeur de types composites
</div>

In [15]:
sql = "UPDATE students SET person=%s WHERE (person).first_name = %s"
c.execute(sql,(('Anna','Conda'),'Elsa'))
conn.commit()

In [16]:
sql = "UPDATE students SET person.last_name = %s WHERE (person).last_name = %s"
c.execute(sql,('Peuplu','Bombeur'))
conn.commit()